In [ ]:
import csv
from datasets import load_dataset, Dataset, concatenate_datasets

In [ ]:
with open('../webui/non_corruption.csv') as csvfile:
    reader = csv.DictReader(csvfile, fieldnames=['url', 'title', 'description', 'text', 'date', 'keywords'])
    rows = list(reader)

In [ ]:
non_corruption_dataset = Dataset.from_list(rows)

In [ ]:
non_corruption_dataset = non_corruption_dataset.map(
    lambda row: {
        'label': 0,
        'is_hand_annoted': True,
        'keywords': row['keywords'].split(',')
    }
)

In [ ]:
non_corruption_dataset

In [ ]:
dataset = load_dataset('boapps/kmdb_classification')

In [ ]:
dataset = dataset.remove_columns(['authors', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'kmonitor_places', 'kmonitor_collections'])

In [ ]:
dataset['train']

In [ ]:
dataset['train'] = concatenate_datasets([dataset['train'], non_corruption_dataset])

In [ ]:
dataset

In [ ]:
dataset.save_to_disk('../datasets/base/kmdb_classification_extended')